In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

print("Package loaded!")

Package loaded!


In [2]:
_data_dir = "./mnist/"

mnist = input_data.read_data_sets(_data_dir, one_hot=True)

img_test = mnist.test.images
label_test = mnist.test.labels

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [3]:
NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE    #784
NUM_HIDDEN1_NODE = 256
NUM_HIDDEN2_NODE = 128

stddev = 0.1

x = tf.placeholder("float", [None, IMAGE_PIXELS])
y = tf.placeholder("float", [None, NUM_CLASSES])

def inference_mlp1(data_in, num_hidden_node):
    with tf.name_scope('hidden'):
        weights = tf.Variable(tf.random_normal([IMAGE_PIXELS, num_hidden_node], stddev=stddev), name='weights')
        biases = tf.Variable(tf.zeros([num_hidden_node]), name='biases')
        hypothesis_1 = tf.matmul(data_in, weights) + b
        gz_1 = tf.sigmoid(hypothesis_1)
        
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(tf.random_normal([num_hidden_node, NUM_CLASSES], stddev=stddev), name='weights')
        biases = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases')
        hypothesis_2 = tf.matmul(gz_1, weights) + biases
    return hypothesis_2

def inference_mlp2(data_in, num_hidden1_node, num_hidden2_node):
    with tf.name_scope('hidden1'):
        weights = tf.Variable(tf.random_normal([IMAGE_PIXELS, num_hidden1_node], stddev=stddev), name='weights')
        biases = tf.Variable(tf.zeros([num_hidden1_node]), name='biases')
        hypothesis_1 = tf.matmul(data_in, weights) + biases
        gz_1 = tf.sigmoid(hypothesis_1)
        
    with tf.name_scope('hidden2'):
        weights = tf.Variable(tf.random_normal([num_hidden1_node, num_hidden2_node], stddev=stddev), name='weights')
        biases = tf.Variable(tf.zeros([num_hidden2_node]), name='biases')
        hypothesis_2 = tf.matmul(gz_1, weights) + biases
        gz_2 = tf.sigmoid(hypothesis_2)
        
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(tf.random_normal([num_hidden2_node, NUM_CLASSES], stddev=stddev), name='weights')
        biases = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases')
        hypothesis_3 = tf.matmul(gz_2, weights) + biases
    return hypothesis_3

def cost(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train = optimizer.minimize(loss, global_step=global_step)
    return train

def evaluation(logits, labels):
    pred = tf.equal(tf.argmax(tf.nn.softmax(logits), 1), tf.argmax(labels, 1))
    accr = tf.reduce_mean(tf.cast(pred, "float"))
    return accr

In [4]:
learning_rate = 0.001

#logits = inference_mlp1(x, NUM_HIDDEN1_NODE)
logits = inference_mlp2(x, NUM_HIDDEN1_NODE, NUM_HIDDEN2_NODE)
loss = cost(logits, y)
train = training(loss, learning_rate=learning_rate)
eval_correct = evaluation(logits, y)

init = tf.initialize_all_variables()

In [5]:
n_samples = mnist.train.num_examples
batch_size = 200
total_batch = int(n_samples / batch_size)
start = time.time()
total_epoch = 20

# SESSION
with tf.Session() as sess:
    sess.run(init)
    # MINI-BATCH LEARNING
    for epoch in range(total_epoch + 1):
        avg_cost = 0.
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed = {x:batch_xs, y:batch_ys}
            sess.run(train, feed)
            avg_cost += sess.run(loss, feed) / batch_size
            
        # DISPLAY
        if epoch % 5 == 0:
            feeds_train = {x:batch_xs, y:batch_ys}
            feeds_test = {x: img_test, y: label_test}
            train_acc = sess.run(eval_correct, feeds_train)
            test_acc = sess.run(eval_correct, feeds_test)
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (epoch, total_epoch, avg_cost, train_acc, test_acc))
print ("DONE")
print time.time() - start

Epoch: 000/020 cost: 1.041465471 train_acc: 0.950 test_acc: 0.912
Epoch: 005/020 cost: 0.161058700 train_acc: 0.970 test_acc: 0.964
Epoch: 010/020 cost: 0.071921972 train_acc: 0.985 test_acc: 0.975
Epoch: 015/020 cost: 0.031656813 train_acc: 0.995 test_acc: 0.978
Epoch: 020/020 cost: 0.013120665 train_acc: 0.995 test_acc: 0.980
DONE
20.6527581215
